In [1]:
import requests

# SPARQL query to fetch all cities in Germany
query = """
SELECT ?city ?cityLabel WHERE {
  ?city wdt:P31/wdt:P279* wd:Q515;
        wdt:P17 wd:Q183.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""

# Wikidata endpoint URL
url = "https://query.wikidata.org/sparql"

# Make the request to the Wikidata SPARQL endpoint
response = requests.get(url, params={'query': query, 'format': 'json'})
data = response.json()
nr=1
# Extract and print city names
for item in data['results']['bindings']:
    nr += 1
    print(nr, item['cityLabel']['value'])

2 Zwickau
3 Plauen
4 Görlitz
5 Neu-Ulm
6 Freital
7 Dippoldiswalde
8 Pirna
9 Sebnitz
10 Freising
11 Dachau
12 Coswig
13 Großenhain
14 Meissen
15 Radebeul
16 Riesa
17 Borna
18 Grimma
19 Markkleeberg
20 Wurzen
21 Delitzsch
22 Eilenburg
23 Oschatz
24 Schkeuditz
25 Torgau
26 Freiberg
27 Bautzen
28 Neumarkt in der Oberpfalz
29 Germering
30 Hoyerswerda
31 Erding
32 Fürstenfeldbruck
33 Deggendorf
34 Forchheim
35 Crimmitschau
36 Glauchau
37 Hohenstein-Ernstthal
38 Limbach-Oberfrohna
39 Werdau
40 Neustadt bei Coburg
41 Weißwasser
42 Weißenburg in Bayern
43 Auerbach
44 Oelsnitz
45 Reichenbach im Vogtland
46 Selb
47 Donauwörth
48 Annaberg-Buchholz
49 Marienberg
50 Schwarzenberg
51 Stollberg
52 Zschopau
53 Brand-Erbisdorf
54 Döbeln
55 Flöha
56 Mittweida
57 Rochlitz
58 Bischofswerda
59 Kamenz
60 Radeberg
61 Neuburg an der Donau
62 Rothenburg ob der Tauber
63 Lindau
64 Löbau
65 Zittau
66 Kulmbach
67 Niesky
68 Eichstätt
69 Traunstein
70 Kitzingen
71 Bad Reichenhall
72 Bad Kissingen
73 Günzburg
74 Nörd

In [1]:
import requests
import datetime
import pandas as pd

def query_wikidata(sparql):
    url = "https://query.wikidata.org/sparql"
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'}
    data = requests.get(url, headers=headers, params={'format': 'json', 'query': sparql})
    return data.json()

def get_us_presidents(last_n_years=80):
    current_year = datetime.datetime.now().year
    start_year = current_year - last_n_years
    
    sparql = """
    SELECT ?president ?presidentLabel ?startTerm ?endTerm WHERE {
        ?president p:P39 ?statement.
        ?statement ps:P39 wd:Q11696; pq:P580 ?startTerm.
        OPTIONAL {?statement pq:P582 ?endTerm.}
        FILTER(YEAR(?startTerm) >= """ + str(start_year) + """)
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    ORDER BY ?startTerm
    """
    return query_wikidata(sparql)

def get_us_economic_data(year, property_code):
    # Common function to get economic data like GDP or unemployment rate
    sparql = f"""
    SELECT ?year ?value WHERE {{
        ?us wdt:{property_code} ?value;
            p:{property_code} [pq:P585 ?date].
        BIND(YEAR(?date) AS ?year)
        FILTER(?year = {year})
        FILTER(?value > 0)
    }}
    LIMIT 1
    """
    results = query_wikidata(sparql)
    data = results.get('results', {}).get('bindings', [])
    if data:
        return data[0]['value']['value']
    return None

def main():
    presidents_data = get_us_presidents()
    data_list = []

    for president in presidents_data['results']['bindings']:
        name = president['presidentLabel']['value']
        start_year = president['startTerm']['value'][:4]
        end_year = president['endTerm']['value'][:4] if 'endTerm' in president else datetime.datetime.now().year
        
        # GDP and unemployment rate queries
        start_gdp = get_us_economic_data(start_year, "P2131")  # GDP
        end_gdp = get_us_economic_data(end_year, "P2131")
        start_unemployment = get_us_economic_data(start_year, "P1198")  # Unemployment rate
        end_unemployment = get_us_economic_data(end_year, "P1198")

        data_list.append({
            "President": name,
            "Start Year": start_year,
            "Start GDP": start_gdp,
            "End Year": end_year,
            "End GDP": end_gdp,
            "Start Unemployment Rate": start_unemployment,
            "End Unemployment Rate": end_unemployment
        })

    df = pd.DataFrame(data_list)
    print(df)
    df.to_csv("us_presidents_economic_data.csv", index=False)

if __name__ == "__main__":
    main()



               President Start Year         Start GDP End Year  \
0        Harry S. Truman       1945              None     1953   
1   Dwight D. Eisenhower       1953              None     1961   
2        John F. Kennedy       1961  23909289978.5861     1963   
3      Lyndon B. Johnson       1963  9136689514.09479     1969   
4          Richard Nixon       1969  23909289978.5861     1974   
5            Gerald Ford       1974   75931656814.657     1977   
6           Jimmy Carter       1977  3012914131.16971     1981   
7          Ronald Reagan       1981  3012914131.16971     1989   
8      George H. W. Bush       1989  3012914131.16971     1993   
9           Bill Clinton       1993  13039352743.9616     2001   
10        George W. Bush       2001  3012914131.16971     2009   
11          Barack Obama       2009  3012914131.16971     2017   
12          Donald Trump       2017  23909289978.5861     2021   
13            Phil Baker       2018         238308749     2024   
14        

In [13]:
import requests
import pandas as pd

def query_wikidata(sparql_query):
    """
    Sends a SPARQL query to the Wikidata endpoint and returns the results as a DataFrame.

    Args:
    sparql_query (str): A SPARQL query string.

    Returns:
    DataFrame: Results of the query as a pandas DataFrame.
    """
    url = "https://query.wikidata.org/sparql"
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0",
        "Accept": "application/sparql-results+json"
    }
    response = requests.get(url, headers=headers, params={'query': sparql_query, 'format': 'json'})
    if response.status_code == 200:
        data = response.json()
        results = data['results']['bindings']
        return pd.json_normalize(results)
    else:
        print(f"Failed to fetch data: HTTP {response.status_code}")
        return pd.DataFrame()

def main():
    # SPARQL query to fetch village name and current mayor in Austria
    sparql_query = """
    SELECT ?village ?villageLabel  WHERE {
      ?village wdt:P31 wd:Q532; # instance of a village
               wdt:P17 wd:Q40. # located in Austria
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    ORDER BY ?villageLabel
    """
    results = query_wikidata(sparql_query)
    if not results.empty:
        print("Data retrieved successfully.")
        print(results.iloc[150:200])
    else:
        print("No data found.")

if __name__ == "__main__":
    main()


Data retrieved successfully.
    village.type                              village.value villageLabel.type  \
150          uri   http://www.wikidata.org/entity/Q69472508           literal   
151          uri   http://www.wikidata.org/entity/Q54854521           literal   
152          uri   http://www.wikidata.org/entity/Q15107518           literal   
153          uri  http://www.wikidata.org/entity/Q118369915           literal   
154          uri   http://www.wikidata.org/entity/Q20826873           literal   
155          uri     http://www.wikidata.org/entity/Q856760           literal   
156          uri   http://www.wikidata.org/entity/Q18616739           literal   
157          uri     http://www.wikidata.org/entity/Q670057           literal   
158          uri     http://www.wikidata.org/entity/Q865764           literal   
159          uri  http://www.wikidata.org/entity/Q118369964           literal   
160          uri   http://www.wikidata.org/entity/Q56223655           literal   